In [1]:
%load_ext autoreload
%autoreload 2
!nvidia-smi

from argparse import Namespace
import sys
import os
home = os.environ['HOME']
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
print(os.environ['CUDA_VISIBLE_DEVICES'])
os.chdir(f'{home}/pycharm/automl')
# os.chdir(f'{home}/pycharm/automl/search_policies/rnn')
sys.path.append(f'{home}/pycharm/nasbench')
sys.path.append(f'{home}/pycharm/automl')
import torch 
import torch.nn as nn

0,1


In [2]:
import utils
from image_dataloader import load_dataset
from search_policies.cnn.search_space.nasbench101.nasbench_api_v2 import NASBench_v2
from search_policies.cnn.search_space.nasbench101.util import load_nasbench_checkpoint, transfer_model_key_to_search_key, nasbench_zero_padding_load_state_dict_pre_hook 
from search_policies.cnn.procedures.train_search_procedure import darts_model_validation
from search_policies.cnn.search_space.nasbench101.model_search import NasBenchNetSearch

Cannot import graphviz package


In [3]:
utils.torch_random_seed(10)
print('loading cifar ...')
train_queue, valid_queue, test_queue = load_dataset()
print("loading Nasbench dataset...")
nasbench = NASBench_v2('data/nasbench/nasbench_only108.tfrecord', only_hash=True)

loading cifar ...
Files already downloaded and verified
Files already downloaded and verified
loading Nasbench dataset...


In [4]:
sanity_check = True

In [30]:
# load one model and 
MODEL_FOLDER='/home/yukaiche/pycharm/automl/experiments/reproduce-nasbench/rank_100002-arch_97a390a2cb02fdfbc505f6ac44a37228-eval-runid-0'
CKPT_PATH=MODEL_FOLDER + '/checkpoint.pt'
model_folder = MODEL_FOLDER
ckpt_path = CKPT_PATH

print("load model and test ...")
hash = model_folder.split('arch_')[1].split('-eval')[0]
print('model_hash ', hash)
spec = nasbench.hash_to_model_spec(hash)
print('model spec', spec)
# model, ckpt = load_nasbench_checkpoint(ckpt_path, spec, legacy=True)

from search_policies.cnn.cnn_search_configs import build_default_args
# def project the weights 
default_args = build_default_args()
default_args.model_spec = spec
spec = nasbench.hash_to_model_spec(hash)
model, ckpt = load_nasbench_checkpoint(ckpt_path, spec, legacy=True)
model = model.cuda()
model.eval()
model_search = NasBenchNetSearch(args=default_args)
model_search.eval()
model_search = model_search.cuda()
source_dict = model.state_dict()
target_dict = model_search.state_dict()

load model and test ...
model_hash  97a390a2cb02fdfbc505f6ac44a37228
model spec Adjacent matrix: [[0, 1, 0, 1, 0, 0, 0], [0, 0, 1, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0]] 
Ops: ['input', 'conv3x3-bn-relu', 'maxpool3x3', 'maxpool3x3', 'maxpool3x3', 'maxpool3x3', 'output']



In [33]:
# Checking the loaded model state dict.
if sanity_check:
    for ind, k in enumerate(source_dict.keys()):
        if ind > 5:
            break
        print(k, ((source_dict[k] - target_dict[k]).sum()))

trans_dict = dict()

for k in model.state_dict().keys():
    kk = transfer_model_key_to_search_key(k, spec)
    if kk not in model_search.state_dict().keys():
        print('not found ', kk)
        continue
    # if sanity_check:
    #     print(f'map {k}', source_dict[k].size()) 
    #     print(f'to {kk}' ,target_dict[kk].size())
    
    trans_dict[kk] = source_dict[k]

padded_dict = nasbench_zero_padding_load_state_dict_pre_hook(trans_dict, model_search.state_dict())

stem.0.weight tensor(3.7924, device='cuda:0')
stem.1.weight tensor(-42.6739, device='cuda:0')
stem.1.bias tensor(5.2125, device='cuda:0')
stem.1.running_mean tensor(0.2281, device='cuda:0')
stem.1.running_var tensor(-110.5854, device='cuda:0')
stem.1.num_batches_tracked tensor(313121, device='cuda:0')


In [36]:
if sanity_check:
    target_dict = model_search.state_dict()
    for ind, k in enumerate(padded_dict.keys()):
        if ind > 5:
            break
        print(k)
        print((padded_dict[k] - target_dict[k]).sum().item())
        print((ckpt['model_state'][k] - target_dict[k]).sum().item())
        # print((padded_dict[k][trans_dict[k].size()[0]] - trans_dict[k]).sum())

stem.0.weight
0.0
0.0
stem.1.weight
0.0
0.0
stem.1.bias
0.0
0.0
stem.1.running_mean
0.0
0.0
stem.1.running_var
0.0
0.0
stem.1.num_batches_tracked
0
0


In [35]:
# loaded the padded dict and test the results.
model_search.load_state_dict(padded_dict)
model_search = model_search.change_model_spec(spec)
model_search = model_search.eval()

In [39]:
res = darts_model_validation(test_queue, model, nn.CrossEntropyLoss(),Namespace(debug=False, report_freq=50))
print('original model evaluation on test split', res)

search_res = darts_model_validation(test_queue, model_search, nn.CrossEntropyLoss(),Namespace(debug=False, report_freq=50))
print('Reload to NasBenchSearch, evaluation results ' ,search_res)
# results get worse after padding to original node. Somethings goes wrong.

original model evaluation on test split (92.95, 0.40452058811187747)
Reload to NasBenchSearch, evaluation results  (92.95, 0.40452058811187747)


In [8]:
# Further sanity checking, by loading part of the model and continue.

In [9]:
# print(model.stacks['stack0']['module0'].vertex_ops.keys())
# print(spec.ops)
# m0 = model.stacks['stack0']['module0']
# print([k for k in m0.state_dict().keys() if 'vertex_1' not in k and 'proj' not in k ])
# # print([k for k in m0.state_dict().keys() if 'vertex_1' not in k and 'proj' not in k ])
# load_keys = sorted([transfer_model_key_to_search_key(k, spec) for k in k1])
# train_param_keys = sorted(train_param_keys)
# # for a1, a2 in zip(load_keys, train_param_keys):
# #     print(a1)
# #     print(a2)
# for a1 in load_keys:
#     if a1 not in train_param_keys:
#         print(a1)
# print(len(load_keys), len(train_param_keys))



In [10]:
model.eval()
model_search.load_state_dict(padded_dict)
model_search.eval()

NasBenchNetSearch(
  (stem): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.997, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (stacks): ModuleDict(
    (stack0): ModuleDict(
      (module0): NasBenchCellSearch(
        (vertex_ops): ModuleDict(
          (vertex_1): MixedVertex(
            (proj_ops): ModuleList(
              (0): DynamicReLUConvBN(
                (relu): ReLU()
                (conv): DynamicConv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (bn): BatchNorm2d(256, eps=1e-05, momentum=0.997, affine=True, track_running_stats=True)
                (channel_drop): ChannelDropout()
              )
            )
            (ops): ModuleDict(
              (conv1x1-bn-relu): DynamicReLUConvBN(
                (relu): ReLU()
                (conv): DynamicConv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
           

In [38]:
# This debugs the entire network's output.
img, lab = iter(test_queue).__next__()
img = img.cuda()
lab = lab.cuda()
bz = 32

with torch.no_grad():
    m1 = model.forward_debug(img[:bz, :, : ,:])
    m2 = model_search.forward_debug(img[:bz, :, :, :])
    for ind,(a,b) in enumerate(zip(m1, m2)):
        print(ind, (a - b).sum().item())

0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0


In [12]:
# model_search.stem[1].eval()
# model.stem[1].eval()
# m1stem_out = model.stem(img)
# model_search.stem.load_state_dict(model.stem.state_dict())
# m2stem_out = model_search.stem(img)
# for k, v in model.stem.state_dict().items():
#     print(k, (v - model_search.stem.state_dict()[k]).sum().item())
# print((m1stem_out - m2stem_out).sum().item())



In [13]:
s1input = m1[0].cuda()
m1stack = model.stacks['stack0']['module0']
m2stack = model_search.stacks['stack0']['module0']
print(m1stack.__class__.__name__)
print(m2stack.__class__.__name__)

print((m1stack(s1input) - m2stack(s1input)).sum().item())

NasBenchCell
NasBenchCellSearch
-29079.44921875


In [14]:
m1out = m1stack.forward_debug(s1input)
m2out = m2stack.forward_debug(s1input)

In [15]:
# Cell level, not true after 1 iters
for k, v in m1out.items():
    print(k, (v - m2out[k]).sum().item())
    
print(m1stack.execution_order.items())

input 0.0
output -29079.44921875
vertex_1 4053.396728515625
vertex_3 -13347.875
vertex_4 -22021.6953125
vertex_5 -28074.6796875
vertex_2 -5058.1689453125
odict_items([('vertex_1', ['input']), ('vertex_3', ['input', 'vertex_1']), ('vertex_4', ['vertex_3']), ('vertex_5', ['vertex_4']), ('vertex_2', ['vertex_1']), ('output', ['vertex_1', 'vertex_2', 'vertex_5'])])


In [16]:
# vertex level
m1vertex = m1stack.vertex_ops['vertex_1']
m2vertex = m2stack.vertex_ops['vertex_1']

print('vertex_1 difference', (m1vertex([s1input,]) - m2vertex([s1input,])).sum().item())
print(m1vertex)
print(m2vertex)

vertex_1 difference 4053.396728515625
Vertex(
  (proj_ops): ModuleList(
    (0): Sequential(
      (0): Conv2d(128, 86, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(86, eps=1e-05, momentum=0.997, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (op): Sequential(
    (0): Conv2d(86, 86, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(86, eps=1e-05, momentum=0.997, affine=True, track_running_stats=True)
    (2): ReLU()
  )
)
MixedVertex(
  (proj_ops): ModuleList(
    (0): DynamicReLUConvBN(
      (relu): ReLU()
      (conv): DynamicConv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.997, affine=True, track_running_stats=True)
      (channel_drop): ChannelDropout()
    )
  )
  (ops): ModuleDict(
    (conv1x1-bn-relu): DynamicReLUConvBN(
      (relu): ReLU()
      (conv): DynamicConv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [17]:
# inside vertex level, proj_ops and vertex_op
m1_proj = m1vertex.proj_ops[0](s1input)
m1_out = m1vertex.op(m1_proj)

m2_proj = m2vertex.current_proj_ops[0](s1input)
m2_out = m2vertex.current_op(m2_proj)

print('after v1 proj diff',(m1_proj - m2_proj).sum().item())
print('after v1 diff',(m1_out - m2_out).sum().item())

after v1 proj diff -12232.4296875
after v1 diff 4053.396728515625


In [18]:
# proj level
m1projop = m1vertex.proj_ops[0]
m2projop = m2vertex.current_proj_ops[0]
print(m1projop)
print(m2projop)

Sequential(
  (0): Conv2d(128, 86, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(86, eps=1e-05, momentum=0.997, affine=True, track_running_stats=True)
  (2): ReLU()
)
DynamicReLUConvBN(
  (relu): ReLU()
  (conv): DynamicConv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(256, eps=1e-05, momentum=0.997, affine=True, track_running_stats=True)
  (channel_drop): ChannelDropout()
)


In [37]:
print(m1projop[1].training)
print(m2projop.bn.training)

False
False


In [20]:
# m2vertex.train()
model_search.eval()
print(m1projop[1].training)
print(m2projop.bn.training)

False
False


In [21]:
import glob
model_lists = glob.glob('/home/yukaiche/pycharm/automl/experiments/reproduce-nasbench/*')
print(model_lists)
for model_folder in model_lists:
    if len(glob.glob(model_folder + '/checkpoint.pt')) > 0:
        hash = model_folder.split('arch_')[1].split('-eval')[0]
        rank = model_folder.split('rank_')[1].split('-arch')[0]
        print(rank, hash)
    



['/home/yukaiche/pycharm/automl/experiments/reproduce-nasbench/rank_300004-arch_f2ff48f97a4f7b340b10f7600a062b5e-eval-runid-0', '/home/yukaiche/pycharm/automl/experiments/reproduce-nasbench/rank_200001-arch_6c678b4d5c0bb786d69ba399c0ee6759-eval-runid-0', '/home/yukaiche/pycharm/automl/experiments/reproduce-nasbench/rank_423000-arch_789eb4f52941c786ac64242606d379fb-eval-runid-0', '/home/yukaiche/pycharm/automl/experiments/reproduce-nasbench/rank_200003-arch_bbbcaeddb4d0782013a4aa63864f46d9-eval-runid-0', '/home/yukaiche/pycharm/automl/experiments/reproduce-nasbench/rank_400001-arch_1119da5413faccc8a6aec77d55d26fc7-eval-runid-0', '/home/yukaiche/pycharm/automl/experiments/reproduce-nasbench/rank_4000-arch_523ac64c85d50533bbbb695d6f3c583f-eval-runid-0', '/home/yukaiche/pycharm/automl/experiments/reproduce-nasbench/rank_5000-arch_81a7f132d0d93078bf6f58ae660985e4-eval-runid-0', '/home/yukaiche/pycharm/automl/experiments/reproduce-nasbench/rank_10-arch_ef11a63e3dec4177d65648771c3689aa-eval-r